# __Handling multiple sequences (PyTorch)__

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

## Модели ожидают пакет входных данных

Последовательности преобразуются в списки чисел. Давайте преобразуем этот список чисел в тензор и отправим его в модель:

In [15]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor(ids)

In [3]:
print(tokens)
print(ids)
print(input_ids)

['i', "'", 've', 'been', 'waiting', 'for', 'a', 'hugging', '##face', 'course', 'my', 'whole', 'life', '.']
[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]
tensor([ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
         2026,  2878,  2166,  1012])


In [ ]:
# This line will fail.
model(input_ids)

#### Ошибка!!!  (IndexError: too many indices for tensor of dimension 1)

<small>у `input_ids` форма (sequence_length,) — одномерный тензор (например, [12, 345, 6789]), а model 
`distilbert-base-uncased-finetuned-sst-2-english` ожидает двумерный тензор с размерностью (batch_size, sequence_length).

<font color='lightgreen'>! Ранее мы использовали модель  
`bert-base-cased` — это оригинальный BERT с torch.nn.Module, который умеет обрабатывать одномерные тензоры (sequence_length,). Модель bert-base-cased реализована так, что если ты передаёшь ей одномерный тензор (sequence_length,), она автоматически добавляет batch размерность внутри forward().  
`distilbert-base-uncased-finetuned-sst-2-english` — это облегчённая версия DistilBERT, оптимизированная для классификации, и она требует (batch_size, sequence_length).</font>

__Как исправить?__  
Добавить размерность batch (размер 1), используя `.unsqueeze(0):`  
`input_ids = torch.tensor(ids).unsqueeze(0)  # Теперь форма (1, sequence_length)`

__Лучший способ – использовать tokenizer()__</small>

In [16]:
tokenized_inputs = tokenizer(sequence, return_tensors="pt")
# print(tokenized_inputs["input_ids"])

Здесь мы можем видеть, что размерность действительно = 1

In [ ]:
# Убираем batch размерность и видим одномерную размерность
tokens_2 = tokenizer(sequence, return_tensors="pt")["input_ids"].squeeze(0)  
print("Input shape:", tokens_2.shape)  # (sequence_length,)

А вот уже `tokenized_inputs` имеет правильный состав и форму и подав его на вход модели ошибка не возникает.

In [17]:
output = model(**tokenized_inputs)
print(output)
print("Logits:", output.logits)

SequenceClassifierOutput(loss=None, logits=tensor([[-1.5607,  1.6123]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
Logits: tensor([[-1.5607,  1.6123]], grad_fn=<AddmmBackward0>)


Попробуем другим способом и он тоже работает, но

In [13]:
tokens_3 = tokenizer.tokenize(sequence)
ids_3 = tokenizer.convert_tokens_to_ids(tokens)
input_ids_3 = torch.tensor([ids_3])
output = model(input_ids_3)
print("Logits:", output.logits)

Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


Вроде бы код
(1)
``` 
tokens_3 = tokenizer.tokenize(sequence)
ids_3 = tokenizer.convert_tokens_to_ids(tokens)
input_ids_3 = torch.tensor([ids_3])
```
делает то-же, что и 
(2)
```
tokenized_inputs = tokenizer(sequence, return_tensors="pt")
```
НО!  
__Различия между inputs_1 и input_2__  
```
Функция	                        (1)	                    (2)
Форма	                        (1, sequence_length)    (1, sequence_length + 2) (из-за [CLS] и [SEP])
Добавляет [CLS] и [SEP]?	❌ Нет                  ✅ Да      
Добавляет attention_mask?	❌ Нет                  ✅ Да	
Готов к использованию в модели?	❌ Нет                  ✅ Да	
```
🎯 Итог:  
`tokenizer(sequence, return_tensors="pt")` = удобный способ, он автоматически добавляет [CLS], [SEP], attention_mask и делает тензор правильной формы.  
Ручная токенизация = ты должен сам добавить `[CLS], [SEP]` и `attention_mask`, иначе модель может работать неправильно.  
Лучше использовать `tokenizer(sequence, return_tensors="pt")`, чтобы избежать ошибок. ✅


### Обработаем входные данные в виде нескольких предложений.

Пакетирование — это процесс отправки нескольких предложений через модель, все сразу.  

In [29]:
sequence_2 = ["I've been waiting for a HuggingFace course my whole life.", 
"I  wait when a HuggingFace will be better.",
"I love HuggingFace"]

Мы должны подать матрицу, или массив данных прямоугольной формы, что преполагает, что все предложения должны быть одинаковой длины. Для этого мы используем `padding=True` в процедуре токенизации.

In [30]:
tokenize_input_2 = tokenizer(sequence_2, padding=True, return_tensors="pt")
tokenize_input_2

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  3524,  2043,  1037, 17662, 12172,  2097,  2022,  2488,
          1012,   102,     0,     0,     0,     0],
        [  101,  1045,  2293, 17662, 12172,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}

In [31]:
output_2 = model(**tokenize_input_2)
output_2.logits

tensor([[-1.5607,  1.6123],
        [ 2.8782, -2.4049],
        [-3.8986,  4.1859]], grad_fn=<AddmmBackward0>)

## Longer sequences (Более длинные последовательности)

Более длинные последовательности
В моделях Transformer существует ограничение на длину последовательностей, которые мы можем передать моделям. Большинство моделей обрабатывают последовательности длиной до `512` или `1024` токенов и будут аварийно завершать работу при запросе на обработку более длинных последовательностей. Есть два решения этой проблемы:

Используйте модель с более длинной поддерживаемой длиной последовательности.
Усекайте последовательности.
Модели имеют разную поддерживаемую длину последовательностей, и некоторые специализируются на обработке очень длинных последовательностей.  
- Longformer — один из примеров, а другой
— LED. Если вы работаете над задачей, требующей очень длинных последовательностей, мы рекомендуем вам взглянуть на эти модели.

В противном случае мы рекомендуем вам усекать последовательности, указав параметр max_sequence_length:

In [ ]:
sequence = sequence[:max_sequence_length]